In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import csv
import numpy as np

DATA_DIR = 'MNIST'
sentiment_data = 'sentiment-data'


def getWordVectorDict():
    reader = csv.reader(open(sentiment_data +'/word-vectors-refine.txt'))

    word_vector_dict = {}
    for row in reader:
        key = row[0]
        if key in word_vector_dict:
            # implement your duplicate row handling here
            pass
        word_vector_dict[key] = np.array(row[1:])
    return word_vector_dict

def getPaddedSentenceMatrix(sentenceMatrix):
    wordCount = 100
    return np.vstack((sentenceMatrix, np.zeros((wordCount - np.shape(sentenceMatrix)[0],np.shape(sentenceMatrix)[1]), dtype=np.float32)))

def getVectorForSentence(sentence, word_vec_dict):
    sentence_matrix = []
    for word in sentence.split(' '):
        word_vec = word_vec_dict[word]
        if(len(sentence_matrix) == 0):
            sentence_matrix = word_vec
        else:
            sentence_matrix = np.vstack((sentence_matrix,word_vec))
    return getPaddedSentenceMatrix(sentence_matrix)

def getData(fileName):
    reader = csv.reader(open(sentiment_data +'/' + fileName))
    trainingData = []
    for row in reader:
        data = {}
        data['label'] =  1 if row[0] == 'postive' else 0
        data['sentence'] = row[1:]
        trainingData.append(data)
    return trainingData

word_vec_dict = getWordVectorDict()


def transform(row):
    return row['label'], getVectorForSentence(row['sentence'][0], word_vec_dict)


word_vector_size = 50;
time_steps = 100;
num_classes = 2
batch_size = 1000;
n_iterations = 10;
hidden_layer_size = 50

training_data = getData('test.csv')
training_rows  = map(lambda row: transform(row), training_data)
training_data = map(lambda row: row[1], training_rows)
training_labels = map(lambda row: row[0], training_rows)
#test_data = map(lambda row: transform(row), getData('test.csv'))
test_data = getData('test.csv')
test_rows  = map(lambda row: transform(row), test_data)
test_data = map(lambda row: row[1], test_rows)
test_labels = map(lambda row: row[0], test_rows)
print training_data[0]

[['0.85051' '1.0817' '-1.4256' ..., '-1.7266' '1.6446' '0.8354']
 ['-0.54182' '-0.60043' '-0.084011' ..., '0.1269' '0.50638' '0.00716']
 ['0.6185' '0.64254' '-0.46552' ..., '-0.27557' '0.30899' '0.48497']
 ..., 
 ['0.26818' '0.14346' '-0.27877' ..., '-0.6321' '-0.25028' '-0.38097']
 ['0.418' '0.24968' '-0.41242' ..., '-0.18411' '-0.11514' '-0.78581']
 ['0.19475' '0.60994' '-0.6662' ..., '-0.71373' '0.23044' '0.83145']]


In [3]:

#Setting up the input and labels placeholders
_inputs = tf.placeholder(tf.float32, shape=[None, time_steps,
                                            word_vector_size])
y = tf.placeholder(tf.int32, shape=[None, num_classes])
y_one_hot = tf.one_hot( y , num_classes )

# TensorFlow built-in functions
# Creating the RNN cell and creating the outputs
with tf.variable_scope("lstm"):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size,
                                             forget_bias=1.0)
    outputs, states = tf.nn.dynamic_rnn(lstm_cell,_inputs, dtype=tf.float32)

weights = {
        'linear_layer': tf.Variable(tf.truncated_normal([hidden_layer_size,
                                                         num_classes],
                                                         mean=0,stddev=.01))
}


# Extract the last relevant output and use in a linear layer
final_output = tf.matmul(states[1],
                         weights["linear_layer"])
softmax = tf.nn.softmax_cross_entropy_with_logits(logits = final_output,
                                                  labels = y)                         
cross_entropy = tf.reduce_mean(softmax)

train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1),
                              tf.argmax(final_output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,
                                   tf.float32)))*100






In [4]:
def dense_to_one_hot(labels_dense, num_classes=10):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

label_batch = np.array(training_labels[100 : 200])
l = np.array(dense_to_one_hot(label_batch,2)).reshape(len(label_batch), num_classes)

print l.shape

(100, 2)


In [9]:
batch_size = 100;

#Initialize session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #Initialize test data
    #test_data = mnist.test.images[:batch_size].reshape((-1,time_steps, word_vector_size))
    #test_label = mnist.test.labels[:batch_size]

    for epoch in range(n_iterations):
        print "Epoch:", epoch
        for j in range(len(training_data)/batch_size):
            #print "j:", j
            if(j == 100):
                break
            startIndex = j*batch_size
            endIndex = startIndex + batch_size
            batch_x = np.array(training_data[startIndex : endIndex]).reshape((-1,time_steps, word_vector_size))
            print len(training_labels[startIndex : endIndex])
            batch_y = dense_to_one_hot(np.array(training_labels[startIndex : endIndex]),num_classes)
            print "X:", len(batch_x)
            print "Y:", len(batch_y)
            #batch_x, batch_y = mnist.train.next_batch(batch_size)
            #batch_x = batch_x.reshape((batch_size, time_steps, word_vector_size))
            sess.run(train_step, feed_dict={_inputs: batch_x,
                                            y: batch_y})
            if epoch % 1 == 0:
                acc = sess.run(accuracy, feed_dict={_inputs: batch_x, y: batch_y})
                loss = sess.run(cross_entropy, feed_dict={_inputs: batch_x, y: batch_y})
                print ("Iter " + str(epoch) + ", Minibatch Loss= " + \
                       "{:.6f}".format(loss) + ", Training Accuracy= " + \
                       "{:.5f}".format(acc))


    print ("Testing Accuracy:",
        sess.run(accuracy, feed_dict={_inputs: test_data, y: test_labels}))

Epoch: 0
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692743, Training Accuracy= 54.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693076, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692974, Training Accuracy= 53.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693594, Training Accuracy= 52.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693057, Training Accuracy= 49.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693049, Training Accuracy= 50.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692576, Training Accuracy= 57.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693146, Training Accuracy= 53.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693389, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692712, Training Accuracy= 54.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692285, Training Accuracy= 61.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692538, Training Accuracy= 55.00000
100
X: 100
Y: 100
Iter 0, Minib

KeyboardInterrupt: 